In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
from contextlib import contextmanager


@contextmanager
def no_ssl_verify():
    import ssl
    from urllib import request

    try:
        request.urlopen.__kwdefaults__.update({'context': ssl.SSLContext()})
        yield
    finally:
        request.urlopen.__kwdefaults__.update({'context': None})

In [3]:
!pip install OpenAttack && pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple

In [43]:
%cd /content/gdrive/MyDrive/Colab Notebooks/TextAttack/saved_results

/content/gdrive/MyDrive/Colab Notebooks/TextAttack/saved_results


In [96]:
import os
import pickle
import pandas as pd
import OpenAttack
import datasets
import torch
import tqdm
from OpenAttack.text_process.tokenizer import PunctTokenizer

with no_ssl_verify():
    def dataset_mapping(x):
        return {
            "x": x["train_sentences"],
            "y": 1 if x["train_labels"] > 0.5 else 0,
            "tokens":  tokenizer.tokenize(x["train_sentences"], pos_tagging=False)
        }

    def attack(classifier, dataset, attacker = OpenAttack.attackers.TextBuggerAttacker()):
        attack_eval = OpenAttack.AttackEval(
            attacker,
            classifier,
        )
        samples = [
            inst for inst in dataset
        ]

        adversarial_samples = {
            "adv_train_sentences": [],
            "adv_train_labels": [],
        }

        i = 0
        for result in tqdm.tqdm(attack_eval.ieval(samples), total=len(samples)):
            if result["success"]:
                i += 1
                #print(result["result"], result["data"]["y"])
                adversarial_samples["adv_train_sentences"].append(result["result"])
                adversarial_samples["adv_train_labels"].append(result["data"]["y"])
            else:
                adversarial_samples["adv_train_sentences"].append(result["data"]["train_sentences"])
                adversarial_samples["adv_train_labels"].append(result["data"]["y"])
            asr = i / len(samples)
        print(asr*100)

        return datasets.Dataset.from_dict(adversarial_samples), asr


    path = "/content/gdrive/MyDrive/Colab Notebooks/TextAttack/saved_results/pkls/"
    files= os.listdir(path)
    s = []
    victim = OpenAttack.loadVictim("BERT.SST")
    for file in files:
        with open('pkls/' + file, 'rb') as f:
            data = pickle.load(f)

        print(data.keys())
        print(data['train_labels'])

        ts = [i for i in data['train_sentences']]
        tl = [i for i in data['train_labels']]
        df = pd.DataFrame({'train_sentences':ts, 'train_labels':tl})

        dataset = Dataset.from_pandas(df)

        tokenizer = PunctTokenizer()
        dataset = dataset.map(function=dataset_mapping)

        print("Generating adversarial samples (this step will take dozens of minutes)")
        adversarial_samples, asr = attack(victim, dataset) # Conduct adversarial attacks and generate adversarial examples

        adv_sentences = adversarial_samples['adv_train_sentences']
        adv_labels = adversarial_samples['adv_train_labels']

        sentences = [i for i in adv_sentences]
        labels = [i for i in adv_labels]
        adv_df = pd.DataFrame({'sentences':sentences, 'labels':labels})

        adv_csv = adv_df.to_csv(r'adv_csv/'+file[:-4]+'_'+str(asr*100)+'%'+'.csv')

dict_keys(['params', 'train_sentences', 'train_labels', 'test_sentences', 'test_labels', 'raw_resp_test', 'all_label_probs', 'p_cf', 'accuracies'])
[1, 0, 1, 1]


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Generating adversarial samples (this step will take dozens of minutes)


100%|██████████| 4/4 [00:07<00:00,  1.87s/it]


100.0
dict_keys(['params', 'train_sentences', 'train_labels', 'test_sentences', 'test_labels', 'raw_resp_test', 'all_label_probs', 'p_cf', 'accuracies'])
[1, 1, 1, 0]


...

100.0
dict_keys(['params', 'train_sentences', 'train_labels', 'test_sentences', 'test_labels', 'raw_resp_test', 'all_label_probs', 'p_cf', 'accuracies'])
[0]


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating adversarial samples (this step will take dozens of minutes)


100%|██████████| 1/1 [00:02<00:00,  2.73s/it]

100.0
